In [ ]:
spark协同推荐，2.4.3版本
https://spark.apache.org/docs/latest/ml-collaborative-filtering.html

In [1]:
import os
import sys
spark_home = os.environ.get('SPARK_HOME',None)
print(spark_home)
sys.path.insert(0,os.path.join(spark_home,'python'))
sys.path.insert(0,os.path.join(spark_home,'python/lib/py4j-0.10.4-src.zip'))

exec(open(os.path.join(spark_home,'python/pyspark/shell.py')).read())

/home/yang0/installer/spark-2.4.3-bin-hadoop2.7
Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.3
      /_/

Using Python version 3.6.5 (default, Jul 11 2019 14:32:02)
SparkSession available as 'spark'.


In [7]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

lines = spark.read.text("./als_data/sample_movielens_ratings.txt").rdd
parts = lines.map(lambda row: row.value.split("::"))
ratingsRDD = parts.map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]),
                                     rating=float(p[2]), timestamp=int(p[3])))
ratings = spark.createDataFrame(ratingsRDD)
(training, test) = ratings.randomSplit([0.8, 0.2])

# 训练
# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop")
model = als.fit(training)

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

# Generate top 10 movie recommendations for each user
# 为所有用户推荐电影
userRecs = model.recommendForAllUsers(10)

# Generate top 10 user recommendations for each movie
# 为所有电影推荐用户
movieRecs = model.recommendForAllItems(10)

# Generate top 10 movie recommendations for a specified set of users
# 三个用户，每个用户十部电影
users = ratings.select(als.getUserCol()).distinct().limit(3)
userSubsetRecs = model.recommendForUserSubset(users, 10)

# Generate top 10 user recommendations for a specified set of movies
# 三部电影，每部前十推荐用户
movies = ratings.select(als.getItemCol()).distinct().limit(3)
movieSubSetRecs = model.recommendForItemSubset(movies, 10)
print(movieSubSetRecs.collect())

Root-mean-square error = 1.4927293118707536
[Row(movieId=65, recommendations=[Row(userId=23, rating=4.835751533508301), Row(userId=12, rating=4.284965515136719), Row(userId=11, rating=3.461029052734375), Row(userId=9, rating=3.2582976818084717), Row(userId=17, rating=2.8085155487060547), Row(userId=29, rating=2.0837039947509766), Row(userId=15, rating=2.053344249725342), Row(userId=26, rating=1.97727370262146), Row(userId=5, rating=1.9405314922332764), Row(userId=3, rating=1.9380824565887451)]), Row(movieId=26, recommendations=[Row(userId=16, rating=3.8295087814331055), Row(userId=10, rating=3.2082841396331787), Row(userId=4, rating=3.1849899291992188), Row(userId=0, rating=2.9807891845703125), Row(userId=15, rating=2.7445921897888184), Row(userId=25, rating=2.538245439529419), Row(userId=11, rating=2.2013213634490967), Row(userId=26, rating=1.8865208625793457), Row(userId=27, rating=1.4297059774398804), Row(userId=29, rating=1.4207522869110107)]), Row(movieId=29, recommendations=[Row(

推荐算法——潜在因子(Latent Factor)算法
https://www.jianshu.com/p/f10d15ac6a80

推荐引擎里面建立标签系统是关键。标签需要有权重，权重最好能变化。